# Sequence to sequence learning for performing number addition

**Author:** [Smerity](https://twitter.com/Smerity) and others<br>
**Date created:** 2015/08/17<br>
**Last modified:** 2020/04/17<br>
**Description:** A model that learns to add strings of numbers, e.g. "535+61" -> "596".

## Introduction

In this example, we train a model to learn to add two numbers, provided as strings.

**Example:**

- Input: "535+61"
- Output: "596"

Input may optionally be reversed, which was shown to increase performance in many tasks
 in: [Learning to Execute](http://arxiv.org/abs/1410.4615) and
[Sequence to Sequence Learning with Neural Networks](

 http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

Theoretically, sequence order inversion introduces shorter term dependencies between
 source and target for this problem.

**Results:**

For two digits (reversed):

+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits (reversed):

+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits (reversed):

+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits (reversed):

+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs


## Setup


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS


## Generate the data


In [63]:
np.random.choice(list("0123456789"))

'6'

In [45]:
np.random.randint(1, DIGITS + 1)

2

In [308]:
f = lambda: int(
    "".join(
        np.random.choice(list("0123456789")) for i in range(np.random.randint(1, DIGITS + 1))
    )
)
print(f())

53


In [333]:

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)
print(ctable.char_indices)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
#     print(a,b)
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
#     print(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
#     print("[%s]"%query)
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
#     print("[%s]"%ans)
    
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))


{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
Generating data...
Total questions: 50000


## Vectorize the data


In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)


Vectorization...


C:\Users\jikim\AppData\Local\Temp/ipykernel_2580/3022509869.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
C:\Users\jikim\AppData\Local\Temp/ipykernel_2580/3022509869.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Build the model


In [334]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Build model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               72192     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 4, 128)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 4, 128)            131584    
                                                                 
 dense_1 (Dense)             (None, 4, 12)             1548      
                                                                 
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## Train the model


In [5]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)



Iteration 1
1407/1407 [==============================] - 28s 17ms/step - loss: 1.7558 - accuracy: 0.3562 - val_loss: 1.5554 - val_accuracy: 0.4196
Q 921+777 T 1698 ☒ 1726
Q 3+432   T 435  ☒ 330 
Q 964+606 T 1570 ☒ 1626
Q 171+186 T 357  ☒ 777 
Q 426+437 T 863  ☒ 707 
Q 14+799  T 813  ☒ 701 
Q 500+5   T 505  ☒ 151 
Q 493+162 T 655  ☒ 777 
Q 42+532  T 574  ☒ 455 
Q 3+867   T 870  ☒ 781 

Iteration 2
1407/1407 [==============================] - 24s 17ms/step - loss: 1.3465 - accuracy: 0.4976 - val_loss: 1.1741 - val_accuracy: 0.5649
Q 182+54  T 236  ☒ 247 
Q 775+776 T 1551 ☒ 1537
Q 674+89  T 763  ☒ 737 
Q 60+256  T 316  ☒ 307 
Q 985+33  T 1018 ☒ 1037
Q 433+32  T 465  ☒ 460 
Q 970+29  T 999  ☒ 900 
Q 74+295  T 369  ☒ 337 
Q 181+94  T 275  ☒ 247 
Q 459+564 T 1023 ☒ 1003

Iteration 3
1407/1407 [==============================] - 25s 18ms/step - loss: 1.0489 - accuracy: 0.6119 - val_loss: 0.9407 - val_accuracy: 0.6611
Q 42+802  T 844  ☑ 844 
Q 35+402  T 437  ☒ 444 
Q 218+920 T 1138 ☒ 1174
Q 6+

1407/1407 [==============================] - 23s 16ms/step - loss: 0.0163 - accuracy: 0.9955 - val_loss: 0.0166 - val_accuracy: 0.9958
Q 398+601 T 999  ☒ 199 
Q 715+785 T 1500 ☑ 1500
Q 159+903 T 1062 ☑ 1062
Q 851+77  T 928  ☑ 928 
Q 325+24  T 349  ☑ 349 
Q 197+423 T 620  ☒ 610 
Q 590+65  T 655  ☑ 655 
Q 149+3   T 152  ☑ 152 
Q 93+306  T 399  ☑ 399 
Q 967+6   T 973  ☑ 973 

Iteration 23
1407/1407 [==============================] - 23s 16ms/step - loss: 0.0179 - accuracy: 0.9951 - val_loss: 0.0314 - val_accuracy: 0.9903
Q 160+181 T 341  ☑ 341 
Q 82+391  T 473  ☑ 473 
Q 166+423 T 589  ☑ 589 
Q 924+19  T 943  ☑ 943 
Q 29+668  T 697  ☑ 697 
Q 53+603  T 656  ☒ 657 
Q 188+39  T 227  ☑ 227 
Q 653+4   T 657  ☑ 657 
Q 86+640  T 726  ☑ 726 
Q 15+200  T 215  ☑ 215 

Iteration 24
1407/1407 [==============================] - 23s 16ms/step - loss: 0.0095 - accuracy: 0.9978 - val_loss: 0.0125 - val_accuracy: 0.9965
Q 997+57  T 1054 ☑ 1054
Q 620+96  T 716  ☑ 716 
Q 885+54  T 939  ☑ 939 
Q 730+819 T 154

You'll get to 99+% validation accuracy after ~30 epochs.
